In [1]:
# Enable autoreload for hot reloading modules
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

# Some how videoseal is added to sys.path at index 0, which causes conflicts
# Remove it to avoid import issues
if "videoseal" in sys.path[0]:
    del sys.path[0]

# We need a specific repo of EditGuard with modification for jittability:
# https://github.com/antoine-tran/EditGuard
sys.path.insert(0, "/checkpoint/avseal/tuantran/env_srcs/editguard/editguard/code")  

sys.path

['/checkpoint/avseal/tuantran/env_srcs/editguard/editguard/code',
 '/checkpoint/avseal/tuantran/envs/lseal/lib/python311.zip',
 '/checkpoint/avseal/tuantran/envs/lseal/lib/python3.11',
 '/checkpoint/avseal/tuantran/envs/lseal/lib/python3.11/lib-dynload',
 '',
 '/checkpoint/avseal/tuantran/envs/lseal/lib/python3.11/site-packages',
 '/checkpoint/avseal/tuantran/omnisealbench/code/src']

In [3]:
import torch

from collections import OrderedDict

import models.networks as networks
from models import create_model

import options.options as option

device = "cuda"

ckpt_path = "/checkpoint/avseal/tuantran/env_srcs/editguard/editguard/checkpoints/clean.pth"
opt = option.parse("/checkpoint/avseal/tuantran/env_srcs/editguard/editguard/code/options/test_editguard.yml", is_train=False)

# model = create_model(opt)
# model.load_test(ckpt)
# model.netG.eval()

net_g = networks.define_G_v2(opt).to(device)
net_g.eval()

ckpt = torch.load(ckpt_path)

load_net_clean = OrderedDict()  # remove unnecessary 'module.'
for k, v in ckpt.items():
    if k.startswith('module.'):
        load_net_clean[k[7:]] = v
    else:
        load_net_clean[k] = v

# Load with strict=False to allow missing 's' buffers (newly added for TorchScript)
net_g.load_state_dict(load_net_clean, strict=False)

export CUDA_VISIBLE_DEVICES=0
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention
SEAttention


_IncompatibleKeys(missing_keys=['irn.operations.0.s', 'irn.operations.1.s', 'irn.operations.2.s', 'irn.operations.3.s', 'irn.operations.4.s', 'irn.operations.5.s', 'irn.operations.6.s', 'irn.operations.7.s', 'irn.operations.8.s', 'irn.operations.9.s', 'irn.operations.10.s', 'irn.operations.11.s'], unexpected_keys=[])

In [6]:
bitencoder = net_g.bitencoder
jit_encoder = torch.jit.script(bitencoder)  #  example_inputs=(torch.randn(1, 3, 256, 256).to(device),)

In [7]:
torch.jit.save(jit_encoder, "/checkpoint/avseal/models/baselines-watermarking/editguard_encoder.pt")

In [8]:
bitdecoder = net_g.bitdecoder
jit_decoder = torch.jit.script(bitdecoder)  # example_inputs=(torch.randn(1

torch.jit.save(jit_decoder, "/checkpoint/avseal/models/baselines-watermarking/editguard_decoder.pt")